<a href="https://colab.research.google.com/github/ankitagithub13/capstone-chatbot/blob/Modeling/Capstone_Concatenated_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simulation for Country and Location 

In [ ]:
Country_loc = {'Country_01':'Local_01', 'Country_01':'Local_03', 'Country_01':'Local_04', 'Country_01':'Local_06', 'Country_01':'Local_11',
             'Country_02':'Local_02', 'Country_02':'Local_05', 'Country_02':'Local_07', 'Country_02':'Local_08', 'Country_02':'Local_09', 'Country_02':'Local_12',
             'Country_03':'Local_10'}

In [ ]:
import random
random.choice(list(Country_loc.items()))

('Country_01', 'Local_11')

Function to extract day and quarter of the week from python

In [ ]:
import datetime
from datetime import timedelta
import math
import pandas as pd

start_date = datetime.date(2021,1,1)
end_date = datetime.date(2021,12,30)

In [ ]:
time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
random_date = start_date + datetime.timedelta(days=random_number_of_days)

In [ ]:
print(random_date)

2021-12-14


In [ ]:
def get_quarter(d):
  return "Q%d" % (math.ceil(d.month/3))
d=random_date
print(get_quarter(d))

Q4


In [ ]:
df = pd.Timestamp(random_date)
print(df.dayofweek)

1


Convolution Model for other parameters

In [119]:
import tensorflow as tf
tf.random.set_seed(100)
import pandas as pd
import numpy as np
from tensorflow import keras 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Embedding, Conv1D, MaxPooling1D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.layers.merge import concatenate

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import tensorflow.experimental.numpy as tnp

In [ ]:
#pip install numpy==1.19

In [ ]:
pip install -U numpy==1.19.2

     |████████████████████████████████| 14.5 MB 4.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
%cd /content/drive/My Drive/Capstone Project

/content/drive/My Drive/Capstone Project


In [5]:
# let's import the dataset and do the preprocessing
input_data=pd.read_csv('Data Set - industrial_safety_and_health_database_with_accidents_description.csv') 

In [6]:
input_data.head()

,Unnamed: 0,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [7]:
# Let's drop the Unnamed column 
input_data1=input_data.drop(columns=['Unnamed: 0'],axis=1)

In [8]:
input_data1.head()

,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [9]:
# As per the exploratory data analysis, we noted that from the time data, only 'day of the week' and 'quarter' had significance, and therefore we will only use those 2 parameters for our model augmentation  

input_data1.Data = pd.to_datetime(input_data1.Data)

input_data1['dayofweek'] =input_data1['Data'].dt.dayofweek
input_data1['quarter'] =input_data1['Data'].dt.quarter

In [10]:
# Let's reorder the data

input_data2=input_data1[['dayofweek', 'quarter','Countries', 'Local','Industry Sector', 'Accident Level', 'Potential Accident Level','Genre', 'Employee or Third Party',
                     'Critical Risk','Description']]

In [11]:
input_data2

,dayofweek,quarter,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,4,1,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,5,1,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,1,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,4,1,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,6,1,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...
...,...,...,...,...,...,...,...,...,...,...,...
420,1,3,Country_01,Local_04,Mining,I,III,Male,Third Party,Others,"Being approximately 5:00 a.m. approximately, w..."
421,1,3,Country_01,Local_03,Mining,I,II,Female,Employee,Others,The collaborator moved from the infrastructure...
422,2,3,Country_02,Local_09,Metals,I,II,Male,Employee,Venomous Animals,During the environmental monitoring activity i...
423,3,3,Country_02,Local_05,Metals,I,II,Male,Employee,Cut,The Employee performed the activity of strippi...


In [13]:
# Although there was some sort of variability caused in the actual accident level by 'day of the week' and 'quarter' as seen in the EDA analysis previously;
# Let's look at the impact of the 'day of the week' and 'quarter' against the potential accident level

dayofweek_table = pd.pivot_table(input_data2, values=['Genre'],index=['dayofweek'], columns=['Potential Accident Level'], 
                             aggfunc=np.count_nonzero, margins=True)

In [15]:
# It seems that there were no link/impact on the potential accident level from the day of the week parameter, although risk seems to be highes in levels 2-4, in day3  
dayofweek_table

Genre                                    
Potential Accident Level     I    II    III     IV     V   VI  All
dayofweek                                                         
0                          3.0  13.0   14.0   22.0   1.0  NaN   53
1                          8.0  19.0   16.0   23.0   4.0  NaN   70
2                         13.0  10.0   18.0   16.0   5.0  1.0   63
3                         11.0  21.0   20.0   23.0   4.0  NaN   79
4                          5.0  14.0   13.0   22.0   8.0  NaN   62
5                          8.0   8.0   13.0   20.0   8.0  NaN   57
6                          1.0  10.0   12.0   17.0   1.0  NaN   41
All                       49.0  95.0  106.0  143.0  31.0  1.0  425

In [17]:
quarter_table = pd.pivot_table(input_data2, values=['Genre'],index=['quarter'], columns=['Potential Accident Level'], 
                             aggfunc=np.count_nonzero, margins=True)

In [18]:
# Again there seems no particular information gain from the quarter feature, though there seems to be a general decrease in risk of accident levels in the 4th quarter, compared to 1st three quarters 

quarter_table

Genre                                    
Potential Accident Level     I    II    III     IV     V   VI  All
quarter                                                           
1                         20.0  24.0   44.0   50.0  15.0  1.0  154
2                         22.0  28.0   36.0   46.0  12.0  NaN  144
3                          5.0  26.0   14.0   22.0   2.0  NaN   69
4                          2.0  17.0   12.0   25.0   2.0  NaN   58
All                       49.0  95.0  106.0  143.0  31.0  1.0  425

In [19]:
# Since we are only going to use 4 variable for the augmented model - Day of week, quarter, Countries and Local - we can drop the other variables

input_data3=input_data2[['dayofweek','quarter','Countries','Local']]

In [20]:
input_data3.head()

,dayofweek,quarter,Countries,Local
0,4,1,Country_01,Local_01
1,5,1,Country_02,Local_02
2,2,1,Country_01,Local_03
3,4,1,Country_01,Local_04
4,6,1,Country_01,Local_04


In [21]:
X1=input_data3

In [28]:
# Let's extract the target variable which is the 
y1=pd.DataFrame(input_data2['Potential Accident Level'])

In [23]:
print(X1.shape)
print(y1.shape)

(425, 4)
(425,)


In [24]:
# let's convert the country and local columns in the dataset into one-hot encoded values before splitting the data
cat_cols=['Countries','Local']
X2= pd.get_dummies(X1, columns = cat_cols)

In [25]:
print(X2.shape)
X2

(425, 17)


,dayofweek,quarter,Countries_Country_01,Countries_Country_02,Countries_Country_03,Local_Local_01,Local_Local_02,Local_Local_03,Local_Local_04,Local_Local_05,Local_Local_06,Local_Local_07,Local_Local_08,Local_Local_09,Local_Local_10,Local_Local_11,Local_Local_12
0,4,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,5,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
421,1,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
422,2,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
423,3,3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [29]:
y1

,Potential Accident Level
0,IV
1,IV
2,III
3,I
4,IV
...,...
420,III
421,II
422,II
423,II


In [77]:
y1['label'] = y1['Potential Accident Level'].replace({'I':0, 'II':1, 'III':2, 'IV':3, 'V':4, 'VI':4})

In [78]:
y=y1['label']

In [79]:
y.shape

(425,)

In [52]:
y2=pd.DataFrame(y1.label)

In [53]:
y2.head()

,label
0,3
1,3
2,2
3,0
4,3


In [54]:
y2['label'].unique()

array([3, 2, 0, 1, 4])

In [55]:
# Since the target labels are non-binary and we will use categorical crossentropy followed by softmax layer to compile the model, we need to convert the label encoded values to categorial using the keras function
from tensorflow.keras.utils import to_categorical

y2 = to_categorical(y2)

In [58]:
y2

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [59]:
y2.shape

(425, 5)

In [60]:
from sklearn.model_selection import train_test_split

In [80]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(X2, y, test_size=0.25, random_state=100)

In [81]:
print(x_train1.shape)
print(x_test1.shape)

(318, 17)
(107, 17)


In [82]:
print(y_train1.shape)
print(y_test1.shape)

(318,)
(107,)


In [83]:
y_train2 = to_categorical(y_train1)
y_test2 =  to_categorical(y_test1)

In [64]:
# We will need to convert the Countries and Local variaables to one-hot encoded values 
x_train1.head()

,dayofweek,quarter,Countries_Country_01,Countries_Country_02,Countries_Country_03,Local_Local_01,Local_Local_02,Local_Local_03,Local_Local_04,Local_Local_05,Local_Local_06,Local_Local_07,Local_Local_08,Local_Local_09,Local_Local_10,Local_Local_11,Local_Local_12
358,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
319,2,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
351,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
361,4,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
404,6,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [65]:
x_test1.head()

,dayofweek,quarter,Countries_Country_01,Countries_Country_02,Countries_Country_03,Local_Local_01,Local_Local_02,Local_Local_03,Local_Local_04,Local_Local_05,Local_Local_06,Local_Local_07,Local_Local_08,Local_Local_09,Local_Local_10,Local_Local_11,Local_Local_12
315,6,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
271,0,4,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
406,6,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
300,4,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
23,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [84]:
y_train2

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [86]:
y_test2

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1.

In [91]:
# We will include 4 aspects - 1) Day of the week, 2) Quarter of the year 3) Country and 4) Location 
# However, since the Categories - Country and Local have been converted to onehot encoded values, the no. of columns have increased from 4 to 17 as seen above 
# Model1


Length =17

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


input_aug= Input(shape=(Length,)) 
input_aug1= tf.expand_dims(input_aug, axis=-1)  #  to add another dimension 

conv1= Conv1D(64, kernel_size=4,activation='relu')(input_aug1)
#drop1=Dropout(0.15)(conv1) ---> did not add any/ much value to the model ac accuracy  
conv2= Conv1D(128, kernel_size=10,activation='relu')(conv1)
#drop2=Dropout(0.15)(conv2)--->did not add any/ much  value to the model 
flat1 = Flatten()(conv2)
dense1= Dense(100,activation='relu')(flat1)
output= Dense(5, activation='softmax')(dense1)

aug_model = Model(inputs=input_aug, outputs=output)
aug_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [89]:
# Let's look at the model summary 
print(aug_model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 17)]              0         
_________________________________________________________________
tf_op_layer_ExpandDims_4 (Te [(None, 17, 1)]           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 14, 64)            320       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 5, 128)            82048     
_________________________________________________________________
flatten_4 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               64100     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 606 

In [75]:
es = EarlyStopping(monitor='val_loss',verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5',save_best_only=True,save_weights_only=True)

In [92]:
history =aug_model.fit(x_train1,y_train2,epochs=200,batch_size=150,validation_data=[x_test1,y_test2],callbacks=[es,mc])

Train on 318 samples, validate on 107 samples
Epoch 1/200
150/318 [=============>................] - ETA: 17s - loss: 1.6202 - acc: 0.1267

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


318/318 [==============================] - 16s 51ms/sample - loss: 1.6063 - acc: 0.2327 - val_loss: 1.5394 - val_acc: 0.4299
Epoch 2/200
318/318 [==============================] - 0s 149us/sample - loss: 1.5394 - acc: 0.4214 - val_loss: 1.4658 - val_acc: 0.4019
Epoch 3/200
318/318 [==============================] - 0s 151us/sample - loss: 1.4707 - acc: 0.4214 - val_loss: 1.3865 - val_acc: 0.4112
Epoch 4/200
318/318 [==============================] - 0s 141us/sample - loss: 1.4067 - acc: 0.4214 - val_loss: 1.3273 - val_acc: 0.4112
Epoch 5/200
318/318 [==============================] - 0s 149us/sample - loss: 1.3654 - acc: 0.4214 - val_loss: 1.2885 - val_acc: 0.4953
Epoch 6/200
318/318 [==============================] - 0s 151us/sample - loss: 1.3262 - acc: 0.4497 - val_loss: 1.2836 - val_acc: 0.4953
Epoch 7/200
318/318 [==============================] - 0s 64us/sample - loss: 1.3071 - acc: 0.4403 - val_loss: 1.2944 - val_acc: 0.4953
Epoch 8/200
318/318 [==============================] -

In [96]:
# let's try a normal Dense model 

Length =17

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


input1_aug= Input(shape=(Length,))

dense1= Dense(50,activation='relu')(input1_aug)
#drop1=Dropout(0.15)(conv1) ---> did not add any/ much value to the model ac accuracy  
dense2= Dense(50,activation='relu')(dense1)
#drop2=Dropout(0.20)(dense2) --->did not add any/ much  value to the model 
#dense3= Dense(100,activation='relu')(dense2) --->did not add any/ much  value to the model
#dense4= Dense(50,activation='relu')(dense2) --->did not add any/ much  value to the model
dense5= Dense(25,activation='relu')(dense2)
#drop2=Dropout(0.30)(dense5) #--->did not add any/ much  value to the model 
output= Dense(5, activation='softmax')(dense5)

aug_model1 = Model(inputs=input1_aug, outputs=output)
aug_model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [94]:
es1= EarlyStopping(monitor='val_loss',verbose=1, patience=75)
mc1= ModelCheckpoint('best_model1.h5',save_best_only=True,save_weights_only=True)

In [97]:
history1 =aug_model1.fit(x_train1,y_train2,epochs=200,batch_size=150,validation_data=[x_test1,y_test2],callbacks=[es1,mc1])

Train on 318 samples, validate on 107 samples
Epoch 1/200
150/318 [=============>................] - ETA: 0s - loss: 1.5790 - acc: 0.2400

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


318/318 [==============================] - 1s 2ms/sample - loss: 1.5809 - acc: 0.2579 - val_loss: 1.5513 - val_acc: 0.3084
Epoch 2/200
318/318 [==============================] - 0s 119us/sample - loss: 1.5550 - acc: 0.2893 - val_loss: 1.5378 - val_acc: 0.2617
Epoch 3/200
318/318 [==============================] - 0s 113us/sample - loss: 1.5354 - acc: 0.3459 - val_loss: 1.5266 - val_acc: 0.3084
Epoch 4/200
318/318 [==============================] - 0s 186us/sample - loss: 1.5185 - acc: 0.3585 - val_loss: 1.5147 - val_acc: 0.3271
Epoch 5/200
318/318 [==============================] - 0s 136us/sample - loss: 1.5034 - acc: 0.3365 - val_loss: 1.5020 - val_acc: 0.3271
Epoch 6/200
318/318 [==============================] - 0s 128us/sample - loss: 1.4896 - acc: 0.3459 - val_loss: 1.4902 - val_acc: 0.3178
Epoch 7/200
318/318 [==============================] - 0s 152us/sample - loss: 1.4774 - acc: 0.3428 - val_loss: 1.4816 - val_acc: 0.3271
Epoch 8/200
318/318 [==============================] - 

Let's add the Bert Model

In [108]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [109]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 12.7 MB/s 


In [98]:

bert_model_name = 'small_bert/bert_en_uncased_L-8_H-768_A-12'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [99]:
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [117]:
pip install bert-tensorflow

In [116]:
import tensorflow  as tf 
tf.enable_eager_execution()

AttributeError: ignored

In [114]:
import tensorflow_hub as hub
import bert
import tokenization
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1', trainable=True)

In [120]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

AttributeError: ignored

In [103]:
#bert_results = bert_layer({'input_word_ids':tf.convert_to_tensor(encoded_text[0], dtype=tf.int32, name='input_word_ids'),
                                                 #'input_mask': tf.convert_to_tensor(encoded_text[1], dtype=tf.int32, name='input_mask'), 
                                                 #'input_type_ids': tf.convert_to_tensor(encoded_text[2], dtype=tf.int32, name='input_type_ids')})

In [104]:
bert_results = bert_layer({'input_word_ids':tf.convert_to_tensor(encoded_text[0], dtype=tf.int32, name='input_word_ids'),
                                                 'input_mask': tf.convert_to_tensor(encoded_text[1], dtype=tf.int32, name='input_mask'), 
                                                 'input_type_ids': tf.convert_to_tensor(encoded_text[2], dtype=tf.int32, name='input_type_ids')})



NameError: ignored

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

    output = bert_layer({'input_word_ids':input_word_ids,
                                                 'input_mask': input_mask, 
                                                 'input_type_ids': segment_ids})
    clf_output = output['pooled_output']
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# let's load the data 
concat_inputdata =pd.read_csv('Data Set - industrial_safety_and_health_database_with_accidents_description.csv')

In [ ]:
# As per the exploratory data analysis, we noted that from the time data, only 'day of the week' and 'quarter' had significance, and therefore we will only use those 2 parameters for our model augmentation  

concat_inputdata.Data = pd.to_datetime(concat_inputdata.Data)

concat_inputdata['dayofweek']=concat_inputdata['Data'].dt.dayofweek
concat_inputdata['quarter']=concat_inputdata['Data'].dt.quarter

In [ ]:
# Let's reorder the data

concat_inputdata1=concat_inputdata[['dayofweek', 'quarter','Countries', 'Local','Industry Sector', 'Accident Level', 'Potential Accident Level','Genre', 'Employee or Third Party',
                     'Critical Risk','Description']]

In [ ]:
concat_inputdata1.head()

,dayofweek,quarter,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,4,1,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,5,1,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,1,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,4,1,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,6,1,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
# we Will need to first split the predictor and label variables; predictor variables can be split into two sets- i.e. 1 with the sentence group and the other with the augmented variables for pre-processing  

X_concat1=pd.DataFrame(concat_inputdata1['Description'])
X_concat2=pd.DataFrame(concat_inputdata1[['dayofweek','quarter','Countries','Local']])
Y1=pd.DataFrame(concat_inputdata1['Potential Accident Level'])

In [ ]:
X_concat1

,Description
0,While removing the drill rod of the Jumbo 08 f...
1,During the activation of a sodium sulphide pum...
2,In the sub-station MILPO located at level +170...
3,Being 9:45 am. approximately in the Nv. 1880 C...
4,Approximately at 11:45 a.m. in circumstances t...
...,...
420,"Being approximately 5:00 a.m. approximately, w..."
421,The collaborator moved from the infrastructure...
422,During the environmental monitoring activity i...
423,The Employee performed the activity of strippi...


In [ ]:
import sys
import os

py_file_location = "/content/drive/My Drive/Capstone Project"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
Y1.head()

,Potential Accident Level
0,IV
1,IV
2,III
3,I
4,IV


In [ ]:
Y1['label'] = Y1['Potential Accident Level'].replace({'I' :0, 'II':1, 'III':2, 'IV':3, 'V':4, 'VI':4})

In [ ]:
Y2=Y1['label']
Y2.head()

0    3
1    3
2    2
3    0
4    3
Name: label, dtype: int64

In [ ]:
from tensorflow.keras.utils import to_categorical
Y2=pd.DataFrame(to_categorical(Y2))
Y2

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
420,0.0,0.0,1.0,0.0,0.0
421,0.0,1.0,0.0,0.0,0.0
422,0.0,1.0,0.0,0.0,0.0
423,0.0,1.0,0.0,0.0,0.0


In [ ]:
import pre_processor
from pre_processor import pre_process_data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
x= pre_process_data()

X_concat1["CleanDescription"] = x.pre_process(X_concat1['Description'], 
                    lower_flag=1,
                    remove_spaces_flag=1, 
                    remove_new_line_flag=1,
                    remove_special_char_flag=1,  
                    remove_numbers_flag=1,  
                    replace_appos_flag=1,
                    remove_stop_words_flag=1,  
                    remove_single_letter_word_flag=1,
                    remove_two_letter_word_flag=1,
                    lemmatize_the_words_flag=0, 
                    stem_the_words_flag=0)

In [ ]:
X_concat1.head()

,Description,CleanDescription
0,While removing the drill rod of the Jumbo 08 f...,removing drill rod jumbo maintenance superviso...
1,During the activation of a sodium sulphide pum...,activation sodium sulphide pump piping uncoupl...
2,In the sub-station MILPO located at level +170...,substation milpo located level collaborator ex...
3,Being 9:45 am. approximately in the Nv. 1880 C...,approximately cx personnel begins task unlocki...
4,Approximately at 11:45 a.m. in circumstances t...,approximately circumstances mechanics anthony ...


In [ ]:
input1_contact=pd.DataFrame(X_concat1.CleanDescription)

In [ ]:
input1_contact.head()

,CleanDescription
0,removing drill rod jumbo maintenance superviso...
1,activation sodium sulphide pump piping uncoupl...
2,substation milpo located level collaborator ex...
3,approximately cx personnel begins task unlocki...
4,approximately circumstances mechanics anthony ...


In [ ]:
# Let's precprocess the other part of the data
# let's convert the country and local columns in the dataset into one-hot encoded values before splitting the data

cat_cols1=['Countries','Local']
input2_contact= pd.get_dummies(X_concat2, columns = cat_cols1)

In [ ]:
input2_contact.head()

,dayofweek,quarter,Countries_Country_01,Countries_Country_02,Countries_Country_03,Local_Local_01,Local_Local_02,Local_Local_03,Local_Local_04,Local_Local_05,Local_Local_06,Local_Local_07,Local_Local_08,Local_Local_09,Local_Local_10,Local_Local_11,Local_Local_12
0,4,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,5,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# let's concatenate the predictor variables and then split the data

input_combined=input1_contact.join(input2_contact)

In [ ]:
input_combined.head()

,CleanDescription,dayofweek,quarter,Countries_Country_01,Countries_Country_02,Countries_Country_03,Local_Local_01,Local_Local_02,Local_Local_03,Local_Local_04,Local_Local_05,Local_Local_06,Local_Local_07,Local_Local_08,Local_Local_09,Local_Local_10,Local_Local_11,Local_Local_12
0,removing drill rod jumbo maintenance superviso...,4,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,activation sodium sulphide pump piping uncoupl...,5,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,substation milpo located level collaborator ex...,2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,approximately cx personnel begins task unlocki...,4,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,approximately circumstances mechanics anthony ...,6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# Let's split the dataset

from sklearn.model_selection import train_test_split

x_train21, X_test21, y_train21, y_test21 = train_test_split(input_combined,Y2, test_size=0.25, random_state=200)

In [ ]:
print(x_train21.shape)
print(X_test21.shape)

(318, 18)
(107, 18)


In [ ]:
print(y_train21.shape)
print(y_test21.shape)

(318, 5)
(107, 5)


In [ ]:
x_train21.columns

Index(['CleanDescription', 'dayofweek', 'quarter', 'Countries_Country_01',
       'Countries_Country_02', 'Countries_Country_03', 'Local_Local_01',
       'Local_Local_02', 'Local_Local_03', 'Local_Local_04', 'Local_Local_05',
       'Local_Local_06', 'Local_Local_07', 'Local_Local_08', 'Local_Local_09',
       'Local_Local_10', 'Local_Local_11', 'Local_Local_12'],
      dtype='object')

In [ ]:
#split the input data again - train

xtrain_input1=x_train21['CleanDescription']
xtrain_input2=x_train21.drop('CleanDescription',axis=1,inplace=False)

In [ ]:
#split the input data again - test

xtest_input1=X_test21['CleanDescription']
xtest_input2=X_test21.drop('CleanDescription',axis=1,inplace=False)

In [ ]:
max_len = 100
train_input =bert_encode(xtrain_input1, tokenizer, max_len=max_len)

In [ ]:
test_input = bert_encode(xtest_input1, tokenizer, max_len=max_len)

In [ ]:
# We will use the vanilla neural network as it does seem to perform slightly better than the conv1d network 

Length2 =17

input2= Input(shape=(Length2,))

dense11= Dense(50,activation='relu')(input2) 
dense21= Dense(50,activation='relu')(dense11)
dense31= Dense(25,activation='relu')(dense21)

In [ ]:
Length1=100
#input1= Input(shape=(Length1,))
input1=[input_word_ids, input_mask, segment_ids]

In [ ]:
from keras.layers.merge import Concatenate

In [ ]:
input_word_ids =tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids =tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

output = bert_layer({'input_word_ids':input_word_ids,'input_mask': input_mask,'input_type_ids': segment_ids})
clf_output = output['pooled_output']

In [ ]:
concat_layer = Concatenate()([clf_output,dense31])
dense_layer= Dense(50, activation='relu')(concat_layer)
output = Dense(5, activation='softmax')(dense_layer)

model = Model(inputs=[input1,input2], outputs=output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
print(model.summary())

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 17)]         0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 50)           900         input_4[0][0]                    
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 100)]        0                                            
___________________________________________________________________________________________

In [ ]:
# the accuracy is not great and lesser than the plan vanilla NN and CONV1d network
# We sill drop the augmented / additional features from the model - i.e. day of the week, quarter, Country and Location

history = model.fit(x=[train_input,xtrain_input2], y=y_train21, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
8/8 [==============================] - 3s 370ms/step - loss: 1.5520 - acc: 0.2559 - val_loss: 1.5970 - val_acc: 0.2344
Epoch 2/20
8/8 [==============================] - 2s 289ms/step - loss: 1.5499 - acc: 0.2992 - val_loss: 1.8848 - val_acc: 0.2344
Epoch 3/20
8/8 [==============================] - 2s 290ms/step - loss: 1.6494 - acc: 0.2953 - val_loss: 1.6876 - val_acc: 0.2344
Epoch 4/20
8/8 [==============================] - 2s 288ms/step - loss: 1.5453 - acc: 0.3346 - val_loss: 1.5364 - val_acc: 0.4062
Epoch 5/20
8/8 [==============================] - 2s 290ms/step - loss: 1.5489 - acc: 0.3346 - val_loss: 1.4821 - val_acc: 0.3906
Epoch 6/20
8/8 [==============================] - 2s 288ms/step - loss: 1.4921 - acc: 0.3110 - val_loss: 1.4641 - val_acc: 0.4219
Epoch 7/20
8/8 [==============================] - 2s 290ms/step - loss: 1.4607 - acc: 0.3504 - val_loss: 1.4567 - val_acc: 0.4531
Epoch 8/20
8/8 [==============================] - 2s 289ms/step - loss: 1.4474 - acc: 0.38